In [18]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
/device:GPU:0
Found GPU at: /device:GPU:0


# Import Library

In [19]:
#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras import Sequential
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from tensorflow.keras.layers import Flatten, Dense,BatchNormalization,Dropout,Input, ZeroPadding2D
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential, Model
import keras.backend as K
from sklearn.metrics import log_loss
# Utils
import os
import cv2
from sklearn.utils import shuffle
from pathlib import Path
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import time

# #mount drive
from google.colab import drive
drive.mount("/content/drive")

#Evaluation
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score


# Core
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns;
from glob import glob 
import random



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Image Augmentation

In [20]:
def img_Generator(file_name,gen_type,size):
    if gen_type=='yes':
        generet_Data=ImageDataGenerator(rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')
        gen_data=generet_Data.flow_from_directory(file_name,target_size=(size, size),
                                                  batch_size=32,shuffle=True,
                                                  class_mode='categorical')
                
    else:
        generet_Data1=ImageDataGenerator()
        gen_data=generet_Data1.flow_from_directory(file_name,target_size=(size, size),
                                                  batch_size=32,shuffle=False,
                                                  class_mode='categorical')
    
    return gen_data

In [21]:
#DenseNet-201
densenet_size=224
image_path=Path('/content/drive/MyDrive/TA/Data/0_3_Split')
data_typ='yes'
train_data=img_Generator(str(image_path)+'/train/',data_typ,densenet_size)
data_typ='no'
test_data=img_Generator(str(image_path)+'/test/',data_typ,densenet_size)

Found 2876 images belonging to 10 classes.
Found 1193 images belonging to 10 classes.


In [22]:
densenet201_base_model = keras.applications.densenet.DenseNet201(include_top=False,weights='imagenet', 
                                                input_tensor=None, input_shape=(densenet_size,densenet_size,3), classes=10)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor="loss", mode="min",min_delta = 0,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [ early, learning_rate_reduction]

In [24]:
print("weights:", len(densenet201_base_model.weights))
print("trainable_weights:", len(densenet201_base_model.trainable_weights))
print("non_trainable_weights:", len(densenet201_base_model.non_trainable_weights))

weights: 1004
trainable_weights: 602
non_trainable_weights: 402


In [25]:
trainable_weight=int(10/100*1004)
print(trainable_weight)

100


In [26]:
#remove few dense layer from model
for layer in densenet201_base_model.layers:
  layer.trainable = False
# for layer in efficientnet_base_model.layers[-183:]:
#     if not isinstance(layer, layers.BatchNormalization) and isinstance(layer, layers.GlobalAveragePooling2D):
#       layer.trainable = True
for layer in densenet201_base_model.layers[-trainable_weight:]:
  layer.trainable = True

In [27]:
print("weights:", len(densenet201_base_model.weights))
print("trainable_weights:", len(densenet201_base_model.trainable_weights))
print("non_trainable_weights:", len(densenet201_base_model.non_trainable_weights))

weights: 1004
trainable_weights: 86
non_trainable_weights: 918


In [28]:
densemodel = Sequential()
densemodel.add(densenet201_base_model)
densemodel.add(layers.Flatten())
# densemodel.add(layers.Dense(1024 ,activation='relu'))
densemodel.add(BatchNormalization())
densemodel.add(Dropout(0.2))
densemodel.add(layers.Dense(10, activation ='softmax'))
densemodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 flatten_1 (Flatten)         (None, 94080)             0         
                                                                 
 batch_normalization_1 (Batc  (None, 94080)            376320    
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 94080)             0         
                                                                 
 dense_1 (Dense)             (None, 10)                940810    
                                                                 
Total params: 19,639,114
Trainable params: 4,710,090
Non-trainable params: 14,929,024
__________________________________

In [29]:
densemodel.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
batch_size = 32
# Training model
n_training_samples = len(train_data)
t0 = time.time()
dense_history = densemodel.fit(
    train_data,
    epochs=200,
    shuffle = True,
    callbacks=callbacks_list
    )
record_time =time.time()-t0
print("Training time:", record_time)

Epoch 1/200
90/90 [==============================] - 1348s 15s/step - loss: 5.9957 - accuracy: 0.3241 - lr: 0.0010
Epoch 2/200
90/90 [==============================] - 43s 480ms/step - loss: 5.9350 - accuracy: 0.4346 - lr: 0.0010
Epoch 3/200
90/90 [==============================] - 43s 481ms/step - loss: 5.3124 - accuracy: 0.4976 - lr: 0.0010
Epoch 4/200
90/90 [==============================] - 44s 489ms/step - loss: 5.2399 - accuracy: 0.5334 - lr: 0.0010
Epoch 5/200
90/90 [==============================] - 43s 478ms/step - loss: 5.3065 - accuracy: 0.5452 - lr: 0.0010
Epoch 6/200
90/90 [==============================] - 44s 485ms/step - loss: 5.0462 - accuracy: 0.5702 - lr: 0.0010
Epoch 7/200
90/90 [==============================] - 44s 481ms/step - loss: 4.5014 - accuracy: 0.6231 - lr: 0.0010
Epoch 8/200
90/90 [==============================] - 43s 473ms/step - loss: 4.7143 - accuracy: 0.6151 - lr: 0.0010
Epoch 9/200
90/90 [==============================] - 44s 487ms/step - loss: 4.39

In [15]:
from keras.models import Model
#save the model
densemodel.save('/content/drive/MyDrive/TA/Models/DenseNet/DenseNet_Adam_10_DropOuts')

In [ ]:
y_val = test_data.classes
y_pred = densemodel.predict(test_data)
y_pred = np.argmax(y_pred, axis=1)
confusion_mtx = confusion_matrix(y_pred, y_val)
f,ax = plt.subplots(figsize=(12, 12))
sns.heatmap(confusion_mtx, annot=True, 
            linewidths=0.01,
            linecolor="white", 
            fmt= '.1f',ax=ax,)
sns.color_palette("rocket")

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
# ax.xaxis.set_ticklabels(test_data.class_indices)
# ax.yaxis.set_ticklabels(rice_classes)
plt.title("Confusion Matrix")
plt.show()

In [17]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_pred, y_val)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_pred, y_val, average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_pred, y_val, average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_pred, y_val, average='macro')
print('F1 score: %f' % f1)

Accuracy: 0.689019
Precision: 0.687770
Recall: 0.688213
F1 score: 0.683104
Accuracy: 0.689019
Precision: 0.687770
Recall: 0.688213
F1 score: 0.683104
